In [4]:
# decision_engine.ipynb

# 1. Setup
from src.rule_engine import RuleEngine
from src.llm_explainer import LLMExplainer
from src.data_generator import FraudDataGenerator
from src.models import FinalDecisionOutput
import pandas as pd

# 2. Generate synthetic data
generator = FraudDataGenerator(fraud_ratio=0.2)
df = generator.generate_dataset(n=50)
df.to_csv("data/sample_transactions.csv", index=False)

# 3. Initialize components
engine = RuleEngine("config/rules_v1.yaml")
explainer = LLMExplainer()  # Uses ANTHROPIC_API_KEY env var

# 4. Process transactions
records = df.to_dict('records')
rule_results = engine.evaluate_batch(records)
explanations = explainer.generate_batch(records, rule_results)

# 5. Combine results
outputs = []
for record, rule_result, explanation in zip(records, rule_results, explanations):
    output = FinalDecisionOutput(
        transaction_id=record['transaction_id'],
        risk_score=rule_result.risk_score,
        decision=rule_result.decision,
        rule_matched=rule_result.matched_rule_name,
        rule_reason=rule_result.rule_reason,
        llm_explanation=explanation.human_readable_explanation,
        confidence=explanation.confidence,
        needs_human_review=explanation.needs_human_review,
        clarifying_questions=explanation.clarifying_questions
    )
    outputs.append(output.model_dump())

results_df = pd.DataFrame(outputs)

# 6. Display results
print(f"Total transactions: {len(results_df)}")
print(f"Flagged for human review: {results_df['needs_human_review'].sum()}")
print(f"\nDecision distribution:")
print(results_df['decision'].value_counts())

# 7. Show items needing review
review_items = results_df[results_df['needs_human_review'] == True]
for _, row in review_items.iterrows():
    print(f"\n--- Transaction {row['transaction_id']} ---")
    print(f"Decision: {row['decision']} (Score: {row['risk_score']})")
    print(f"Explanation: {row['llm_explanation']}")
    print(f"Questions: {row['clarifying_questions']}")

Total transactions: 50
Flagged for human review: 14

Decision distribution:
decision
Decision.ALLOW     40
Decision.BLOCK      6
Decision.REVIEW     4
Name: count, dtype: int64

--- Transaction 615ffcae ---
Decision: Decision.ALLOW (Score: 10)
Explanation: This £166.28 electronics purchase was approved because it shows no significant fraud indicators. The transaction occurred in the UK matching the account's home country, involves a mature account (5+ years old), and falls within normal spending patterns with only 2 transactions in the past 24 hours.
Questions: ["What type of device was used for this transaction and does it match the customer's typical login pattern?", "Given this is flagged as a new device, what authentication methods were used to verify the customer's identity?"]

--- Transaction 2cb72737 ---
Decision: Decision.ALLOW (Score: 10)
Explanation: This €144.27 travel transaction was approved because it shows normal customer behavior with no significant fraud indicators. Th